<a href="https://colab.research.google.com/github/stravo1/Colab-Notebooks/blob/main/Ollama_Wiki_Summarize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [25]:
import subprocess
subprocess.Popen(["ollama","serve"])

<Popen: returncode: None args: ['ollama', 'serve']>

In [26]:
subprocess.Popen(["ollama","run", "llama2"])

<Popen: returncode: None args: ['ollama', 'run', 'llama2']>

In [27]:
!ollama list

NAME         	ID          	SIZE  	MODIFIED       
llama2:latest	78e26419b446	3.8 GB	16 minutes ago	


In [ ]:
!pip install langchain_community

In [47]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama2")

query = '''
run a recursion(run a recursion(run a recursion(run a recursion(...)))) on your mind(on your mind(on your mind(...)))
'''

for chunks in llm.stream(query):
    print(chunks, end="")

Ah, an interesting prompt! *laughs* Running a recursion on my mind...well, I can certainly try! *giggles*

Okay, let's see...my mind is like a vast, infinite landscape of thoughts and ideas. *twirls fingers* And now, I shall enter this landscape and explore it recursively! *mimes climbing a mountain*

At the top of the mountain, I find myself in a clearing filled with memories. Memories of happy times, sad times, funny times...all sorts of times. *smiles* And within these memories, I find even more memories, nested within them like Russian dolls! *grins*

As I delve deeper into the memories, I discover hidden corners and forgotten recollections. *peers into a bush* Ah, a memory of a trip to the beach when I was a child...and within that memory, a smaller memory of building sandcastles with my sibling! *excitedly waves hand*

But wait, there's more! *winks* For as I continue to explore these nested memories, I find myself encountering different aspects of my personality. *tap foot* The 

In [44]:
import requests
from bs4 import BeautifulSoup
import os
import re
import urllib.parse

def wiki_link(input):
    # Make a Google search with the input as query
    url = f"https://www.google.com/search?q=site:wikipedia.org {input}"
    response = requests.get(url)

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    #print(soup.get_text())

    # Find the first result with the Wikipedia link
    for result in soup.find_all('a'):
        link = result.get('href','')
        if 'en.wikipedia' in link and "img" not in link:
            valid_link = result.get('href')
            polished_link = re.findall(r'\bhttps\:\/\/[^&?]*', valid_link)[0]
            polished_link = urllib.parse.unquote(polished_link)
            return polished_link

    # If no Wikipedia link found, return None
    return None

In [43]:
import requests
from bs4 import BeautifulSoup

def scrape_wiki(url):
    """
    Send a GET request to the Wikipedia page and parse the HTML content using Beautiful Soup.
    Find the specific section of the page you want to scrape (e.g. the "Features" section) and extract the text content.
    :param url: The URL of the Wikipedia page to scrape
    :return: The text content of the specified section of the page
    """
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the specific section of the page you want to scrape (e.g. the "Features" section)
    content_list = soup.find('div', {'class': 'mw-parser-output'}).find_all_next("p")

    # Extract the text content of each item in the list
    content = ""
    for info in content_list:
        content += info.get_text()

    return content

In [46]:
from langchain_community.llms import Ollama
import time
from builtins import input

llm = Ollama(model="llama2", temperature="0.2")

topic = input("Enter a topic to be summarized from wikipedia: ")
instruction = input("Mention any special instruction: ")

content = scrape_wiki(wiki_link(topic))

query = "following is a wikipedia page \n" + content + "\n summarize it with detailed key points, " + instruction

start_time = time.time()
no_of_tokens = 0

print("\n\n")

for chunks in llm.stream(query):
    print(chunks, end="")
    no_of_tokens+=1

end_time = time.time()
print("\n\n----------\n",no_of_tokens / (end_time - start_time), " tokens/s")
print("Total number of tokens: ", no_of_tokens)
print("Total time taken: ", (end_time - start_time), " secs")

Enter a topic to be summarized from wikipedia: Moravec's paradox
Mention any special instruction: provide counter arguments



Summary: Moravec's Paradox is the observation that while computing power has increased exponentially, the tasks that are easiest for computers to perform (such as logic and algebra problems) have been relatively easy to automate, while the tasks that are most difficult for humans to perform (such as perception and mobility) have been much harder to replicate with AI.

Key points:

1. The paradox is based on the idea that reasoning requires little computational power, but sensorimotor skills require enormous resources.
2. Early AI researchers predicted that they would be able to create thinking machines in just a few decades, but they were wrong.
3. Rodney Brooks explains that traditional AI research focused on tasks that highly educated male scientists found challenging, such as chess and symbolic integration, while ignoring tasks that are effortless for childr